In [1]:
import sys
import os
import re
import yaml
import json
import pandas as pd
import numpy as np
from pyparsing import *

from data_model import *

# Sobre

Notebook experimental sobre extrair dados do powerbi via formato pbip e tmdl

# Referências

* [TMDL model no powerbi](https://powerbi.microsoft.com/en-us/blog/tmdl-in-power-bi-desktop-developer-mode-preview/#:~:text=Before%20giving%20it%20a%20try,semantic%20model%20using%20TMDL%20format%E2%80%9D.)

# Extrair dados do TMLD

In [2]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return lines

In [ ]:
def parse_tmdl_groups(lines):
    groups = []
    current_group = []
    current_calc = []
    current_comments = []
    for line in lines:
        if not line.strip():
            continue   # Ignora as linhas em branco
        if line.startswith('///') and not current_group:
            current_comments.append(line.strip().strip('/// '))
            continue
        elif line.startswith('///'):
            current_group_temp = [current_group[0]] + [cur_line.removeprefix('\t') for cur_line in current_group[1:]]
            prop = ''.join(current_group_temp[1:])
            if '\t' in prop:
                prop = parse_tmdl_groups([line + '\n' for line in prop.split('\n')])
                current_calc = []
            tmdl = TMDL.create(description=current_comments, 
                        element=current_group_temp[0], 
                        properties=prop,
                        calculation=current_calc)
            groups.append(tmdl)
            current_group = []
            current_calc = []
            current_comments = [line.strip().strip('/// ')]
            continue

        if line[0] == '\t'and not line[1] == '\t':
            current_group.append(line)
        elif line[0] == '\t' and line[1] == '\t':
            current_calc.append(line)
            current_group.append(line)
        else:
            if current_group: 
                current_group_temp = [current_group[0]] + [cur_line.removeprefix('\t') for cur_line in current_group[1:]]
                prop = ''.join(current_group_temp[1:])
                if '\t' in prop:
                    prop = parse_tmdl_groups([line + '\n' for line in prop.split('\n')])
                    current_calc = []
                tmdl = TMDL.create(description=current_comments, 
                                   element=current_group_temp[0], 
                                   properties=prop,
                                   calculation=current_calc)
                groups.append(tmdl)
                current_group = []
                current_comments = []
                current_calc = []
            current_group.append(line.strip())
    current_group_temp = [current_group[0]] + [cur_line.removeprefix('\t') for cur_line in current_group[1:]]
    prop = ''.join(current_group_temp[1:])
    if '\t' in prop:
        prop = parse_tmdl_groups([cur_line + '\n' for cur_line in prop.split('\n')])
        current_calc = []
    tmdl = TMDL.create(description=current_comments, 
                        element=current_group_temp[0], 
                        calculation=current_calc,
                        properties=prop)
    groups.append(tmdl)
    return groups

In [211]:
file_content = 'pbip/PNP_Publicada_dev.SemanticModel/definition/tables/NaturezaDespesa.tmdl'
file_content = 'pbip/PNP_Publicada_dev.SemanticModel/definition/tables/dimCurso.tmdl'
file_content = 'pbip/PNP_Publicada_dev.SemanticModel/definition/model.tmdl'

lines = read_file(file_content)
parsed_data = parse_tmdl_groups(lines)


here
here
here
here
here


In [208]:
lines

['/// @description A tabela dimCurso possui informações sobre os cursos oferecidos por uma instituição de ensino.\n',
 '/// @author dsbr\n',
 '/// @version 1\n',
 '/// @date 03/11/2022\t\n',
 'table dimCurso\n',
 '\tlineageTag: e6e8bb82-8093-4629-9276-4f9d480108e8\n',
 '\n',
 '\t/// @description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.\n',
 '\t/// @author dsbr\n',
 '\t/// @version 1\n',
 '\t/// @date 03/11/2022\n',
 "\tmeasure 'Matrícula Equivalente' = ```\n",
 '\t\t\t\n',
 '\t\t\t/*@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.\n',
 '\t\t\t@author dsbr\n',
 '\t\t\t@version 1\n',
 '\t\t\t@date 03/11/2022*/\n',
 '\t\t\t\n',
 '\t\t\t\n',
 '\t\t\tCALCULATE\n',
 '\t\t\t([Matrícula Equivalente Apresentada Geral], dimFinanciamento[nomeFonteFinanciamentoCorrigido] IN {"Sem Programa Associado", "MedioTec"})\n',
 '\t\t\t```\n',


In [194]:
parsed_data

[TMDL(description=['@description A tabela dimCurso possui informações sobre os cursos oferecidos por uma instituição de ensino.', '@author dsbr', '@version 1', '@date 03/11/2022'], element='table dimCurso', properties=[TMDL(description=[], element='lineageTag: e6e8bb82-8093-4629-9276-4f9d480108e8', properties='', calculation=[]), TMDL(description=['@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.', '@author dsbr', '@version 1', '@date 03/11/2022'], element="measure 'Matrícula Equivalente' = ```", properties=[TMDL(description=[], element='\t/*@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.\n', properties='@author dsbr\n@version 1\n@date 03/11/2022*/\nCALCULATE\n([Matrícula Equivalente Apresentada Geral], dimFinanciamento[nomeFonteFinanciamentoCorrigido] IN {"Sem Programa Associado", "MedioTec"})\n```\n', calculation=

In [195]:
parsed_data[0].element

'table dimCurso'

In [196]:
parsed_data[0].properties

[TMDL(description=[], element='lineageTag: e6e8bb82-8093-4629-9276-4f9d480108e8', properties='', calculation=[]),
 TMDL(description=['@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.', '@author dsbr', '@version 1', '@date 03/11/2022'], element="measure 'Matrícula Equivalente' = ```", properties=[TMDL(description=[], element='\t/*@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.\n', properties='@author dsbr\n@version 1\n@date 03/11/2022*/\nCALCULATE\n([Matrícula Equivalente Apresentada Geral], dimFinanciamento[nomeFonteFinanciamentoCorrigido] IN {"Sem Programa Associado", "MedioTec"})\n```\n', calculation=[]), TMDL(description=[], element='formatString: 0.000', properties='', calculation=[]), TMDL(description=[], element='lineageTag: d67dd037-af45-4c8c-b957-a21e69aedb2b', properties='', calculation=[]), TMDL(descriptio

In [197]:
parsed_data[0].calculation

[]

In [198]:
parsed_data[0].properties[1].description

['@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.',
 '@author dsbr',
 '@version 1',
 '@date 03/11/2022']

In [199]:
parsed_data[0].properties[-2]

TMDL(description=[], element='partition dimCurso-9656090d-95e9-402e-a238-20bb37c17853 = m', properties=[TMDL(description=[], element='mode: import', properties='', calculation=[]), TMDL(description=[], element="queryGroup: 'Modelo PNP\\Dimensões'", properties='', calculation=[]), TMDL(description=[], element='source =', properties=[TMDL(description=[], element='\tlet\n', properties=[TMDL(description=[], element='\tFonte = Csv.Document(Binary.Decompress(Web.Contents(AWS&"dimCurso.csv.gz"),Compression.GZip),[Delimiter=";", Columns=15, Encoding=1252, QuoteStyle=QuoteStyle.None]),\n', properties='#"Cabeçalhos Promovidos" = Table.PromoteHeaders(Fonte, [PromoteAllScalars=true]),\n#"Tipo Alterado" = Table.TransformColumnTypes(#"Cabeçalhos Promovidos",{{"codCurso", Int64.Type}, {"codUnidade", Int64.Type}, {"tipoEixoTecnologico", type text}, {"ModalidadeEnsino", type text}, {"nomeCurso", type text}, {"SubeixoTecnologico", type text}, {"tipoCurso", type text}, {"tipoOferta", type text}, {"turnoC

In [200]:
parsed_data[0].properties[-2].element

'partition dimCurso-9656090d-95e9-402e-a238-20bb37c17853 = m'

In [201]:
parsed_data[0].properties[-2].properties

[TMDL(description=[], element='mode: import', properties='', calculation=[]),
 TMDL(description=[], element="queryGroup: 'Modelo PNP\\Dimensões'", properties='', calculation=[]),
 TMDL(description=[], element='source =', properties=[TMDL(description=[], element='\tlet\n', properties=[TMDL(description=[], element='\tFonte = Csv.Document(Binary.Decompress(Web.Contents(AWS&"dimCurso.csv.gz"),Compression.GZip),[Delimiter=";", Columns=15, Encoding=1252, QuoteStyle=QuoteStyle.None]),\n', properties='#"Cabeçalhos Promovidos" = Table.PromoteHeaders(Fonte, [PromoteAllScalars=true]),\n#"Tipo Alterado" = Table.TransformColumnTypes(#"Cabeçalhos Promovidos",{{"codCurso", Int64.Type}, {"codUnidade", Int64.Type}, {"tipoEixoTecnologico", type text}, {"ModalidadeEnsino", type text}, {"nomeCurso", type text}, {"SubeixoTecnologico", type text}, {"tipoCurso", type text}, {"tipoOferta", type text}, {"turnoCurso", type text}, {"CHMinimaCurso", Int64.Type}, {"fatorEsforcoCurso", type number}, {"skCursoMatric

In [204]:
parsed_data[0].properties[1].calculation

[]

In [144]:
parsed_data[0].calculation

['\t\t\t/*@description Corresponde à matrícula ponderada pelos fatores de: - Fator de Equiparação de carga horária; - Fator de Esforço de curso.\n',
 '\t\t\t@author dsbr\n',
 '\t\t\t@version 1\n',
 '\t\t\t@date 03/11/2022*/\n',
 '\t\t\tCALCULATE\n',
 '\t\t\t([Matrícula Equivalente Apresentada Geral], dimFinanciamento[nomeFonteFinanciamentoCorrigido] IN {"Sem Programa Associado", "MedioTec"})\n',
 '\t\t\t```\n',
 '\t\tformatString: 0.000\n',
 '\t\tlineageTag: d67dd037-af45-4c8c-b957-a21e69aedb2b\n',
 '\t\tchangedProperty = FormatString\n',
 '\t\tdataType: int64\n',
 '\t\tisHidden\n',
 '\t\tformatString: 0\n',
 '\t\tlineageTag: b60cedce-2e16-448a-aae8-9357ae50a21c\n',
 '\t\tsummarizeBy: sum\n',
 '\t\tsourceColumn: codCurso\n',
 '\t\tchangedProperty = IsHidden\n',
 '\t\tannotation SummarizationSetBy = Automatic\n',
 '\t\tdataType: int64\n',
 '\t\tisHidden\n',
 '\t\tformatString: 0\n',
 '\t\tlineageTag: 7171ce63-0635-40c4-83c1-6f200adfb676\n',
 '\t\tsummarizeBy: sum\n',
 '\t\tsourceColumn:

In [1]:
import json
from jinja2 import Environment, FileSystemLoader

In [2]:
with open("tables.json", "r", encoding="utf-8") as f:
    tmdl_data = json.load(f)

In [4]:
for file_name, tmdl_list in tmdl_data.items():
    for tmdl_obj in tmdl_list:
        print(tmdl_obj.get("element", ""))

table @project
table Atualizacao
table Brasao
table contador-monitor
table 'Contexto Orçamentário'
table 'Controle Empenho'
table DateTableTemplate_efd75c12-63d8-4a48-bff0-810439d01e27
table dCalendInterval
table dCalendário
table dimCargos
table dimCarreiras
table dimCicloMatricula
table dimCurso
table dimEscolaridade
table dimFinanciamento
table dimFuncao
table dimJornada
table dimOrgao
table dimRendaRaca
table dimSexoIdade
table dimSexoIdadeSiape
table dimSitFuncional
table dimSituacao
table dimTempoServico
table dimUnidade
table dimUsernameInstagram
table dimVagas
table d_IES
table Esfera
table fCiclos
table fCPC
table fEficienciaAcademica
table fFinanceiro
table fFuncoes
table fIGC
table fInstagram
table Fonte
table fOrçamentoComparar
table 'fServidores | Microdados'
table fServidoresSiape
table fSituaçãoMatrícula
table FuncFinanceiras
table fVagas
table icons
table IndicAcompOrc
table IndicadorVar
table ItemDespesa
table LocalDateTable_031d51da-ffa4-4d99-9903-68ebf2059f1b
table L